<h2> Imports </h2>

In [2]:
import pandas as pd
import numpy as np
import json
import os

<h1> Step 1: Prepare data and configure the environment </h1>

<h3> 1. Data Uploading (Data I/O) </h3>

1.1. Loading data from files

In [4]:
# Define the base directories
dataset_dir = '../Data_Generator/datasets'
topology_dir = '../Topology_Generator/topologies'

Loading dataset

In [13]:
# Initialize dictionaries for datasets and topologies
datasets = {}

# Function to load datasets from a directory
def load_datasets_from_directory(dataset_dir):
    
    # Loop through each folder in the datasets directory
    for dataset_name in os.listdir(dataset_dir):
        dataset_path = os.path.join(dataset_dir, dataset_name)
        
        # Only process directories
        if os.path.isdir(dataset_path):
            episodes = os.path.join(dataset_path, "episodes.csv")
            agents = os.path.join(dataset_path, "agents.csv")
            arrivals = os.path.join(dataset_path, "arrivals.csv")
            tasks = os.path.join(dataset_path, "tasks.csv")
            
            dataset = {}
            dataset['episodes'] = pd.read_csv(episodes)
            dataset['agents'] = pd.read_csv(agents)
            dataset['arrivals'] = pd.read_csv(arrivals)
            dataset['tasks'] = pd.read_csv(tasks)
            datasets[dataset_name] = dataset
                
    # return datasets


In [29]:
# Call the functions to load data and topologies
load_datasets_from_directory(dataset_dir)

print("dataset heavy -> agents")
display(datasets['heavy']['agents'].head())
datasets['heavy']['agents'].info()

print("\ndataset heavy -> arrivals")
display(datasets['heavy']['arrivals'].head())
datasets['heavy']['arrivals'].info()

print("\ndataset heavy -> episodes")
display(datasets['heavy']['episodes'].head())
datasets['heavy']['episodes'].info()

print("\ndataset heavy -> tasks")
display(datasets['heavy']['tasks'].head())
datasets['heavy']['tasks'].info()

heavy -> agents


,agent_id,f_local,m_local,lam_sec
0,0,1.741183e+09,5.713850e+09,0.708673
1,1,1.352326e+09,4.566429e+09,0.234989
2,2,1.726668e+09,5.815120e+09,0.228174
3,3,1.543616e+09,3.539850e+09,0.310369
4,4,1.130883e+09,4.161368e+09,0.548990


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   agent_id  18 non-null     int64  
 1   f_local   18 non-null     float64
 2   m_local   18 non-null     float64
 3   lam_sec   18 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 704.0 bytes

heavy -> arrivals


,scenario,episode_id,t_slot,t_time,agent_id,task_id
0,heavy,0,0,0.0,0,0
1,heavy,0,0,0.0,1,1
2,heavy,0,0,0.0,4,2
3,heavy,0,0,0.0,7,3
4,heavy,0,0,0.0,10,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   scenario    30636 non-null  object 
 1   episode_id  30636 non-null  int64  
 2   t_slot      30636 non-null  int64  
 3   t_time      30636 non-null  float64
 4   agent_id    30636 non-null  int64  
 5   task_id     30636 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 1.4+ MB

heavy -> episodes


,scenario,episode_id,Delta,T_slots,hours,N_agents,seed
0,heavy,0,1.0,3600,1.0,18,345


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   scenario    1 non-null      object 
 1   episode_id  1 non-null      int64  
 2   Delta       1 non-null      float64
 3   T_slots     1 non-null      int64  
 4   hours       1 non-null      float64
 5   N_agents    1 non-null      int64  
 6   seed        1 non-null      int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 184.0+ bytes

heavy -> tasks


,scenario,episode_id,task_id,agent_id,t_arrival_slot,t_arrival_time,b_mb,rho_cyc_per_mb,c_cycles,mem_mb,modality,has_deadline,deadline_s,deadline_time,non_atomic,split_ratio,action_space_hint
0,heavy,0,0,0,0,0.0,5.000003,1.499999e+09,7.499998e+09,64.000006,sensor,1,0.800726,0.800726,0,0.000000,discrete
1,heavy,0,1,1,0,0.0,5.000001,1.500000e+09,7.499999e+09,64.000031,image,1,0.615113,0.615113,0,0.000000,discrete
2,heavy,0,2,4,0,0.0,5.000005,1.500002e+09,7.500016e+09,64.000021,text,1,0.323007,0.323007,1,0.539704,continuous
3,heavy,0,3,7,0,0.0,5.000002,1.500001e+09,7.500006e+09,63.999980,sensor,1,0.481587,0.481587,0,0.000000,discrete
4,heavy,0,4,10,0,0.0,5.000008,1.499999e+09,7.500006e+09,64.000022,sensor,1,0.594564,0.594564,0,0.000000,discrete


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   scenario           30636 non-null  object 
 1   episode_id         30636 non-null  int64  
 2   task_id            30636 non-null  int64  
 3   agent_id           30636 non-null  int64  
 4   t_arrival_slot     30636 non-null  int64  
 5   t_arrival_time     30636 non-null  float64
 6   b_mb               30636 non-null  float64
 7   rho_cyc_per_mb     30636 non-null  float64
 8   c_cycles           30636 non-null  float64
 9   mem_mb             30636 non-null  float64
 10  modality           30636 non-null  object 
 11  has_deadline       30636 non-null  int64  
 12  deadline_s         10673 non-null  float64
 13  deadline_time      10673 non-null  float64
 14  non_atomic         30636 non-null  int64  
 15  split_ratio        30636 non-null  float64
 16  action_space_hint  306

Loading topology

In [40]:
topologies = {}

def load_topologies_from_directory(topology_dir):
    
    # Loop through each folder in the topologies directory
    for topology_name in os.listdir(topology_dir):
        topology_path = os.path.join(topology_dir, topology_name)
        # print("topology_path", topology_path)
        
        # Only process directories
        if os.path.isdir(topology_path):
            topology_json_path = os.path.join(topology_path, "topology.json")
            meta_json_path = os.path.join(topology_path, "topology_meta.json")
            connection_matrix_csv_path = os.path.join(topology_path, "connection_matrix.csv")
            
             # --- Load JSON files ---
            topology_data = None
            meta_data = None
            # if os.path.exists(topology_json_path):
            with open(topology_json_path, "r", encoding="utf-8") as f:
                topology_data = json.load(f)
            # if os.path.exists(meta_json_path):
            with open(meta_json_path, "r", encoding="utf-8") as f:
                meta_data = json.load(f)
            
            # Load the connection matrix CSV file
            # if os.path.exists(connection_matrix_csv):
            connection_matrix = pd.read_csv(connection_matrix_csv_path)
            # else:
                # connection_matrix = None  # Handle case if the file doesn't exist
            
            # Store the topology details and the loaded CSV
            topologies[topology_name] = {
                "topology_json": topology_data,
                "meta_json": meta_data,
                # "connection_matrix_csv": connection_matrix_csv,
                "connection_matrix": connection_matrix  # Store the loaded CSV data
            }
    
    # return topologies

In [42]:
load_topologies_from_directory(topology_dir)

print('topology clustered -> connection_matrix')
display(topologies['clustered']['connection_matrix'].head())
topologies['clustered']['connection_matrix'].info()

print('topology clustered -> topology_json')
print(topologies['clustered']['topology_json'])

print('topology clustered -> meta_json')
print(topologies['clustered']['meta_json'])


topology clustered -> connection_matrix


,no.,mec_0no.,mec_1no.,mec_2no.,mec_3no.,mec_4no.,mec_5no.,mec_6no.,mec_7no.,mec_8no.,mec_9no.,mec_10no.,mec_11no.,mec_12no.,mec_13no.,mec_14no.,mec_15no.,mec_16no.,mec_17no.,cloud
0,mec_0,0.000000,9.798801,11.958295,11.470404,9.980195,8.814050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.912800
1,mec_1,9.798801,0.000000,8.375143,11.535600,10.702584,9.136893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.609202
2,mec_2,11.958295,8.375143,0.000000,8.814632,9.637714,9.170700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.055348
3,mec_3,11.470404,11.535600,8.814632,0.000000,10.377054,11.920252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.851664
4,mec_4,9.980195,10.702584,9.637714,10.377054,0.000000,8.105383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.076301


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   no.        18 non-null     object 
 1   mec_0no.   18 non-null     float64
 2   mec_1no.   18 non-null     float64
 3   mec_2no.   18 non-null     float64
 4   mec_3no.   18 non-null     float64
 5   mec_4no.   18 non-null     float64
 6   mec_5no.   18 non-null     float64
 7   mec_6no.   18 non-null     float64
 8   mec_7no.   18 non-null     float64
 9   mec_8no.   18 non-null     float64
 10  mec_9no.   18 non-null     float64
 11  mec_10no.  18 non-null     float64
 12  mec_11no.  18 non-null     float64
 13  mec_12no.  18 non-null     float64
 14  mec_13no.  18 non-null     float64
 15  mec_14no.  18 non-null     float64
 16  mec_15no.  18 non-null     float64
 17  mec_16no.  18 non-null     float64
 18  mec_17no.  18 non-null     float64
 19  cloud      18 non-null     float64
dtypes: float64(1

This code loads five input files and returns the data in the form of data frames (for CSV files) and dictionaries (for JSON files).